# Vortex center glow

The aim of this simulation is to model the vortex center glow effect, which is caused to diffraction of the thermal emission from the environment outside the entrance pupil to the pupil image formed on the detector, by the vortex phase mask. 
To model the effect, several points outside the entrance pupil are taken and their intensity on the detector is evaluated. Net effect is determined by adding contributions from all these points.

### Imports

In [ ]:
import heeps
from heeps.util.coord import cart_coord, polar_coord
from heeps.util.img_processing import resize_img, crop_img, pad_img
from heeps.optics import fp_mask, lyot_stop, detector
from copy import deepcopy
import math
import proper
import os.path
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

### Create a spot

In [ ]:
def spot(npupil, rad, theta, r_spot):
    x, y = cart_coord(npupil)
    c = (rad*np.cos(np.deg2rad(theta)), rad*np.sin(np.deg2rad(theta)))
    r =  np.abs((x-c[0]) + 1j*(y-c[1]))
    circ = (r < r_spot)
    return circ

### VCG simulations

In [ ]:
conf = dict(
    band = 'L',
    mode = 'CVC',
    dir_output = 'vcg',
    file_lyot_stop = '',
    cpu_count = None
    )
# 1. Create a config dictionary with your simulation parameters in read_config
conf = heeps.config.read_config(verbose=False, **conf)
npupil = conf['npupil']
ngrid = conf['ngrid']
diam_ext = conf['diam_ext']
pupil_img_size = conf['pupil_img_size']
lam = conf['lam']

# 2. Update config parameters. 
conf = heeps.config.update_config(saveconf=True, verbose=True, **conf)
conf['ls_dRint'] = 0.0291
conf['ls_dRext'] = 0.0291
conf['ls_dRspi'] = 0.0317

# 3. Load cold stop
cold_stop = fits.getdata('input_files/pupil/cold_stop.fits')
cold_stop = resize_img(cold_stop, npupil)
cold_stop = pad_img(cold_stop, ngrid)

# 4. Initialize wavefront using PROPER
beam_ratio = npupil/ngrid*(diam_ext/pupil_img_size)
wf = proper.prop_begin(pupil_img_size, lam, ngrid, beam_ratio)

# 5. VCG from points inside central obscuration
vcg_in = []
for rad in list(np.arange(0.0, 0.28, 0.005)):
    for theta in np.arange(0,360,10):
        print(rad, theta)
        # keep a copy of the input wavefront
        wf1 = deepcopy(wf)
        # create entrance pupil
        r_spot = 0.0025                           # specify spot size
        pup = spot(10526, rad, theta, r_spot)     # create a spot
        pup = resize_img(pup, npupil)             # resize to npupil
        pup = pad_img(pup, ngrid)                 # pad with zeros
        # normalize the entrance pupil intensity (total flux = 1)
        I_pup = pup**2
        pup = np.sqrt(I_pup/np.sum(I_pup))
        # add to wavefront
        proper.prop_multiply(wf1, pup)
        # propagate one frame of onaxis psf (i.e. star)
        proper.prop_multiply(wf1, cold_stop)      # apply cold stop
        wf1 = fp_mask(wf1, verbose=True, **conf)
        wf1 = lyot_stop(wf1, verbose=True, **conf)
        psf = detector(wf1, verbose=True, **conf)
        # scale the psf
        psf = [((i/36)*(4*rad))/(r_spot) for i in psf]
        vcg_in.append(psf)

# 6. VCG from points beyond ELT pupil's outer edge
vcg_out = []
for rad in list(np.arange(0.92, 1.0, 0.005)):
    for theta in np.arange(0,360,5):
        print(rad, theta)
        # keep a copy of the input wavefront
        wf1 = deepcopy(wf)
        # create entrance pupil
        r_spot = 0.0025                           # specify spot size
        pup = spot(10526, rad, theta, r_spot)     # create a spot
        pup = resize_img(pup, npupil)             # resize to npupil
        pup = pad_img(pup, ngrid)                 # pad with zeros
        # normalize the entrance pupil intensity (total flux = 1)
        I_pup = pup**2
        pup = np.sqrt(I_pup/np.sum(I_pup))
        # add to wavefront
        proper.prop_multiply(wf1, pup)
        # propagate one frame of onaxis psf (i.e. star)
        proper.prop_multiply(wf1, cold_stop)       # apply cold stop
        wf1 = fp_mask(wf1, verbose=True, **conf)
        wf1 = lyot_stop(wf1, verbose=True, **conf)
        psf = detector(wf1, verbose=True, **conf)
        # scale the psf
        psf = [((i/72)*(4*rad))/(r_spot) for i in psf]
        vcg_out.append(psf)

# 7. VCG from points along spiders
vcg_spi = []
for rad in list(np.arange(0.28, 0.92, 0.005)):
    for theta in np.arange(30,360,60):
        print(rad, theta)
        # keep a copy of the input wavefront
        wf1 = deepcopy(wf)
        # create entrance pupil
        r_spot = 0.0025                           # specify spot size
        pup = spot(10526, rad, theta, r_spot)     # create a spot
        pup = resize_img(pup, npupil)             # resize to npupil
        pup = pad_img(pup, ngrid)                 # pad with zeros
        # normalize the entrance pupil intensity (total flux = 1)
        I_pup = pup**2
        pup = np.sqrt(I_pup/np.sum(I_pup))
        # add to wavefront
        proper.prop_multiply(wf1, pup)
        # propagate one frame of onaxis psf (i.e. star)
        proper.prop_multiply(wf1, cold_stop)       # apply cold stop
        wf1 = fp_mask(wf1, verbose=True, **conf)
        wf1 = lyot_stop(wf1, verbose=True, **conf)
        psf = detector(wf1, verbose=True, **conf)
        # scale the psf
        psf = [(i*(2*50))/(math.pi*20) for i in psf]
        vcg_spi.append(psf)

vcg_inner = sum(np.array(vcg_in))
vcg_outer = sum(np.array(vcg_out))
vcg_spiders = sum(np.array(vcg_spi))
vcg_net = sum(np.array(vcg_in) + np.array(vcg_out) + np.array(vcg_spi))
plt.imshow(vcg_net, origin='lower')
fits.writeto('VCG_inner.fits', np.float32(vcg_inner), overwrite=True)
fits.writeto('VCG_outer.fits', np.float32(vcg_outer), overwrite=True)
fits.writeto('VCG_spiders.fits', np.float32(vcg_spiders), overwrite=True)
fits.writeto('VCG.fits', np.float32(vcg_net), overwrite=True)